In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.

# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

# Loading a pre-trained model in inference mode

In this tutorial, we will show how to extract features in inference mode from a VISSL pre-trained trunk.

We will concentrate on loading and extracting features from a SimCLR model. 
This tutorial, however, is portable to any another pre-training methods (MoCo, SimSiam, SwAV, etc). See [here](https://github.com/facebookresearch/vissl/blob/main/MODEL_ZOO.md) for a list of the models in our model zoo. 

Through it, we will show:

1. How to instantiate a model with a pre-training configuration
2. Load the weights of the pre-trained model from our model zoo.
3. Use it to extract the TRUNK features associated with the VISSL Logo

**NOTE:** For a tutorial focused on how to use VISSL to schedule a feature extraction job, please refer to [the dedicated tutorial](https://colab.research.google.com/github/facebookresearch/vissl/blob/stable/tutorials/Feature_Extraction.ipynb).

**NOTE:** Please ensure your Collab Notebook has GPU available: `Edit -> Notebook Settings -> select GPU`.

**NOTE:** You can make a copy of this tutorial by `File -> Open in playground mode` and make changes there. Please do NOT request access to this tutorial.


## Install VISSL

We will start this tutorial by installing VISSL, following the instructions [here](https://github.com/facebookresearch/vissl/blob/main/INSTALL.md#install-vissl-pip-package).

In [ ]:
# Install pytorch version 1.8
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install Apex by checking system settings: cuda version, pytorch version, and python version
import sys
import torch
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{torch.__version__[0:5:2]}"
])
print(version_str)

# install apex (pre-compiled with optimizer C++ extensions and CUDA kernels)
!pip install apex -f https://dl.fbaipublicfiles.com/vissl/packaging/apexwheels/{version_str}/download.html

# # clone vissl repository and checkout latest version.
!git clone --recursive https://github.com/facebookresearch/vissl.git

%cd vissl/

!git checkout v0.1.6
!git checkout -b v0.1.6

# install vissl dependencies
!pip install --progress-bar off -r requirements.txt
!pip install opencv-python

# update classy vision install to commit compatible with v0.1.6
!pip uninstall -y classy_vision
!pip install classy-vision@https://github.com/facebookresearch/ClassyVision/tarball/4785d5ee19d3bcedd5b28c1eb51ea1f59188b54d

# Update fairscale to commit compatible with v0.1.6
!pip uninstall -y fairscale
!pip install fairscale@https://github.com/facebookresearch/fairscale/tarball/df7db85cef7f9c30a5b821007754b96eb1f977b6

# install vissl dev mode (e stands for editable)
!pip install -e .[dev]

VISSL should be successfuly installed by now and all the dependencies should be available.

In [1]:
import vissl
import tensorboard
import apex
import torch

/opt/conda/envs/vissl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading a VISSL SimCLR pre-trained model




## Download the ResNet-50 Simclr weights from the Model Zoo

In [2]:
!wget -q -O resnet_simclr.torch https://dl.fbaipublicfiles.com/vissl/model_zoo/simclr_rn50_1000ep_simclr_8node_resnet_16_07_20.afe428c7/model_final_checkpoint_phase999.torch

## Create the model associated to the configuration

Load the configuration and merge it with the default configuration.

In [3]:
from omegaconf import OmegaConf
from vissl.utils.hydra_config import AttrDict

from vissl.utils.hydra_config import compose_hydra_configuration, convert_to_attrdict

# Config is located at vissl/configs/config/pretrain/simclr/simclr_8node_resnet.yaml.
# All other options override the simclr_8node_resnet.yaml config.

cfg = [
  'config=pretrain/simclr/simclr_8node_resnet.yaml',
  'config.MODEL.WEIGHTS_INIT.PARAMS_FILE=resnet_simclr.torch', # Specify path for the model weights.
  'config.MODEL.FEATURE_EVAL_SETTINGS.EVAL_MODE_ON=True', # Turn on model evaluation mode.
  'config.MODEL.FEATURE_EVAL_SETTINGS.FREEZE_TRUNK_ONLY=True', # Freeze trunk. 
  'config.MODEL.FEATURE_EVAL_SETTINGS.EXTRACT_TRUNK_FEATURES_ONLY=True', # Extract the trunk features, as opposed to the HEAD.
  'config.MODEL.FEATURE_EVAL_SETTINGS.SHOULD_FLATTEN_FEATS=False', # Do not flatten features.
  'config.MODEL.FEATURE_EVAL_SETTINGS.LINEAR_EVAL_FEAT_POOL_OPS_MAP=[["res5avg", ["Identity", []]]]' # Extract only the res5avg features.
]

# Compose the hydra configuration.
cfg = compose_hydra_configuration(cfg)
# Convert to AttrDict. This method will also infer certain config options
# and validate the config is valid.
_, cfg = convert_to_attrdict(cfg)

** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



And then build the model:

In [4]:
from vissl.models import build_model

model = build_model(cfg.MODEL, cfg.OPTIMIZER)

In [5]:
model

Classy <class 'vissl.models.base_ssl_model.BaseSSLMultiInputOutputModel'>:
BaseSSLMultiInputOutputModel(
  (_heads): ModuleDict()
  (trunk): FeatureExtractorModel(
    (base_model): ResNeXt(
      (_feature_blocks): ModuleDict(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1_relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           

## Loading the pre-trained weights

In [6]:
from classy_vision.generic.util import load_checkpoint
from vissl.utils.checkpoint import init_model_from_consolidated_weights

# Load the checkpoint weights.
weights = load_checkpoint(checkpoint_path=cfg.MODEL.WEIGHTS_INIT.PARAMS_FILE)


# Initializei the model with the simclr model weights.
init_model_from_consolidated_weights(
    config=cfg,
    model=model,
    state_dict=weights,
    state_dict_key_name="classy_state_dict",
    skip_layers=[],  # Use this if you do not want to load all layers
)

print("Weights have loaded")

Weights have loaded


## Extracting the Trunk Output on the VISSL Logo.

In [7]:
!wget -q -O /content/test_image.jpg https://raw.githubusercontent.com/facebookresearch/vissl/master/.github/logo/Logo_Color_Light_BG.png

/content/test_image.jpg: No such file or directory


new function with image directly instead of path 

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

def extract_features(img):
  #image = Image.open(path)

  # Convert images to RGB. This is important
  # as the model was trained on RGB images.
  image = img.convert("RGB")

  pipeline = transforms.Compose([
      transforms.Resize(224),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711)),
  ])
  x = pipeline(image)

  features = model(x.unsqueeze(0))
  
  return torch.squeeze(features[0])

# PASCAL VOC 

In [ ]:
from torchvision import datasets
import torch
import numpy as np
from torch.utils.data import ConcatDataset
import os

In [ ]:
trainval_dataset_2012 = datasets.VOCDetection(root="/home/ubuntu/mmdetection_od/mmdetection/data", year="2012", image_set="trainval", download=False)

In [ ]:
len(trainval_dataset_2012)

In [ ]:
trainval_dataset_2007 =  datasets.VOCDetection(root="/home/ubuntu/mmdetection_od/mmdetection/data",year="2007",image_set="trainval", download=False)

In [ ]:
len(trainval_dataset_2007)

In [ ]:
train_dataset = ConcatDataset([trainval_dataset_2007, trainval_dataset_2012])


In [ ]:
len(train_dataset)

# Train embeddings

In [ ]:
image_features = []
image_file_names=[]

# iterate over the dataset 
for idx in range(len(train_dataset)):
    # get the image and its annotations
    img, target = train_dataset[idx]

    image_features.append(extract_features(img))
    image_file_names.append(target['annotation']['filename'])

image_features

In [ ]:
image_features[0].shape

In [ ]:
len(image_features)

In [ ]:
image_features_cpu=[image_feature.cpu().numpy() for image_feature in image_features]

# Save features

In [ ]:
#Save features in the correct results folder
dataset='pascalvoc'
results_folder = os.path.join('/home/ubuntu/master_thesis/covering_lens/TypiClust/scan','results')
if not os.path.exists(results_folder):
        os.makedirs(results_folder)
results_folder_dataset = os.path.join(results_folder,dataset)
if not os.path.exists(results_folder_dataset):
        os.makedirs(results_folder_dataset)
results_folder_dataset_pretext = os.path.join(results_folder_dataset,'pretext')
if not os.path.exists(results_folder_dataset_pretext):
        os.makedirs(results_folder_dataset_pretext)
np.save(os.path.join(results_folder_dataset_pretext,'features_seed1_simclr'), image_features_cpu)

In [ ]:
check_emb=np.load('/home/ubuntu/master_thesis/covering_lens/TypiClust/scan/results/pascalvoc/pretext/features_seed1_simclr.npy')

In [ ]:
check_emb.shape

In [ ]:
len(image_file_names)

In [ ]:
with open(os.path.join(results_folder_dataset_pretext, 'filenames.txt'), 'w') as f:
    for image_file_name in image_file_names:
        f.write(image_file_name + '\n')